In [ ]:
import json
from slackclient import SlackClient

# Read the access token from the file
secrets_file = 'slack_secret.json'
f = open(secrets_file, 'r') 
content = f.read()
f.close()
auth_info = json.loads(content)
token = auth_info["access_token"]
sc = SlackClient(token)


In [ ]:
import pandas as pd
from pandas.io.json import json_normalize

In [ ]:
# Get channel list and store in database
response = sc.api_call("channels.list")
df_channels = json_normalize(response["channels"])
df_channels.drop('members', axis=1, inplace = True)
# df_channels.to_sql(name='channels', con=mysql, if_exists='replace')
df_channels

In [ ]:
# Get users list and store in database
response = sc.api_call("users.list")
df_users = json_normalize(response["members"])
# df_users.to_sql(name='users', con=engine, if_exists='replace')
df_users

In [ ]:
# Retrieves all past messages from all public channels
def get_messages(channel_id, latest_fetched_ts): 
    oldest = sys.maxsize
    results = []
    while True:
        response = sc.api_call("channels.history", channel=channel_id, latest=oldest)
        data = [{"timestamp": entry.get("ts"), "user": entry.get("user"), "message": entry.get("text")} for entry in response["messages"] ]
        results.extend(data)
        oldest = min([entry["ts"] for entry in response["messages"]])
        latest = max([entry["ts"] for entry in response["messages"]])
        if response["has_more"] == False:
            break
    df = pd.DataFrame(results)
    return df

In [ ]:
import sys

for i, row in df_channels[["id", "name"]].iterrows():
    channel_id = row["id"]
    channel_name = row["name"]
    print(channel_id, channel_name)
    df_msgs = get_messages(channel_id, sys.maxsize)
    df_msgs["channel_id"] = row["id"]
    # df_msgs.to_sql(name='messages', con=engine, if_exists='append')